## 1. Importing software packages and loading Baseline

For running this tutorial, you need to have:
- The `message_ix` modeling framework was installed successfully with a version >= 3.4.0 (see instructions [here](https://docs.messageix.org/en/stable/#getting-started) ).
- Updated your code by pulling modifications done for this study from [this fork](https://github.com/behnam-zakeri/message_ix/tree/central-asia-paper).

*Notice*: If you have installed `message_ix` from source, you can "pull" changes from the abovementioned fork automatically. If you have installed `message_ix` using Anaconda/miniconda and you are not familiar with `git`, you need to update your `message_ix` code manually to be able to run this tutorial. For doing so, you need to copy the following folder and file from [this fork](https://github.com/behnam-zakeri/message_ix/tree/central-asia-paper) and replace them in your `message_ix` installation:
- `message_ix/model/MESSAGE` [folder](https://github.com/behnam-zakeri/message_ix/tree/central-asia-paper/message_ix/model/MESSAGE), and
- `message_ix/models.py` [file](https://github.com/behnam-zakeri/message_ix/tree/central-asia-paper/message_ix/models.py).

In [ ]:
# Importing required software packages
import pandas as pd
import os
import ixmp
import message_ix
from plotter import plotter
from postprocessor import yearly_plot, cost_emission_plot

In [ ]:
# Getting path to this project on your machine
# path = os.path.dirname(os.path.realpath("__file__")).strip("\scripts")
path = os.getcwd().strip("\scripts")

In [ ]:
# Loading modeling platform (this will link you to your database)
mp = ixmp.Platform("ixmp-dev")

In [ ]:
# Creating a new scenario with desired model/scenario names
model = 'MESSAGEix_KR'
scenario = 'baseline'
base = message_ix.Scenario(mp, model, scenario, version='new')

In [ ]:
# Specify the filename to the input Excel file
filename = '\\MESSAGEix_KR_baseline.xlsx'
xls_file = path + "\\data" + filename

# Loading data of the Baseline scenario from Excel (may take few minutes)
# base.read_excel("H:\MyDocuments\Projects\GUIDE\MESSAGEix_KR_collabo\data\MESSAGEix_KR_baseline.xlsx", init_items=True, add_units=True, commit_steps=True)
base.read_excel(xls_file, init_items=True, add_units=True, commit_steps=True)

## 2. Looking into model parameters (input data)
`message_ix` formulation is written in GAMS, a programming and optimization software. Hence, `message_ix` uses the same syntax as GAMS, which means the model has:
- index sets: such as set of technologies ('technology'), locations ('node'), etc.
- parameters (input data): such as investment cost ('inv_cost'), capcaity factor ('capacity_factor') etc.
- variables (output results): such as total installed capacity ('CAP'), activity ('ACT') etc.

You can list these items by using relevant syntax. For example, for listing `message_ix` sets, you can run the command: `base.set_list()`, and for listing available parameters run `base.par_list()`.
For more information on each of the model sets and parameters, please refer to the documentation, e.g., [parameter definitions](https://docs.messageix.org/en/master/model/MESSAGE/parameter_def.html). 

### Hint:
You can use python syntax to filter or look for some specific parameters, e.g., containing a word or term in their name (see example below).

In [ ]:
# Listing parameters that have a string of "cost" in their name
[x for x in base.par_list() if 'cost' in x]

### 2.1. Loading input data
You can load parameters by their name and investigate them. The output will be a python dataframe, which looks like a stacked table. Also, we can load a portion of a parameter data, i.e., by slicing the table. See the examples below:

In [ ]:
# Loading data of a parameter (with no filter, it returns the entire table)
parname = 'var_cost'   # variable O&M cost
table = base.par(parname)
# see the table and column names (if the table is large, you will only see the beginning and end rows)
table

In [ ]:
# Loading data of a parameter, by filtering a portion of the table
parname = 'inv_cost'   # CAPEX
tec = 'coal_ppl'       # coal power plant
node = 'Korea'           # Korea

# Notice the syntax for filtering
table = base.par(parname, {'technology': tec, 'node_loc':node})
# see the table and column names
table

In [ ]:
# Loading data of a parameter, even with more filters
parname = 'var_cost'   # variable O&M cost
techno = 'coal_extr'   # coal extraction process
node = 'Korea'           # Uzbekistan
years = [2025, 2030, 2035, 2040]           # activity years of 2025-2040

# Notice the syntax when we pass a list to an index
table = base.par(parname, {'technology': techno, 'node_loc':node,
                           'year_act': years})
# see the content
table  

In [ ]:
# If you don't know the index names related to the parameter (i.e., column names of the table), you can see them by:
base.idx_names('inv_cost')

### Info (1): units in the MESSAGEix_KR model
As you saw in the example above, "unit" is not listed as an index. In the MESSAGEix formulation, "unit" is defined by the user just for information, and it does NOT affect the calculations. In other words, MESSAGEix does not check the units, nor convert them. Hence, this is the user that should be careful when defining parameters and adding "value" to the parameters to ensure consistency and accuracy of the units. In the MESSAGEix_KR model, the units are as follows:
- unit of capacity: GW
- unit of activity: GWa (gigawatts-year) (1 GWa = 8.76 TWh)
- unit of investment cost: \\$/kW (= M\\$/GW)
- unit of variable cost: \\$/kWa (= M\\$/GWa)
- unit of water flow: 1000 m3/s (thousand cubic meter per second)
- unit of CO2 emission: Mt of Ceq
- unit of other emissions: kt

### 2.2. Visualizing input data
You can visualize input data and investigate them. For this, you can use python visualization methods or a few functions provided in this repository.

In [ ]:
# Visualizing input data in a parameter for certain technologies or commodities
node = "Korea"
tec = "solar_pv_ppl"        # solar PV
parameter = "inv_cost"      # CAPEX
df = base.par(parameter, {"technology": tec, "node_loc": node}).sort_values(["year_vtg"])
df = df.loc[df["year_vtg"] >= 2015]

# Plotting
pl = pd.Series(df["value"].to_list(),
                        index=pd.Index(df["year_vtg"].to_list(), name='Year'))
pl.plot(title='CAPEX of {} ($/kW)'.format(tec), ylim=[0, 1500])

## 4. Solving the scenario
After committing the changes, we are ready to solve the scenario. Solving the scenario means sending the input data from your database to GAMS formulation in the form of GDX files, solving the mathematical formulation, and retrieving the output results (MESSAGEix variables) back to your database. If solving the scenario succeeds without an error, we will be able to look into the results. Otherwise, you need to find out why the scenario has been insolvable.

#### Tip (1): Solver options
Large MESSAGEix models solve more quickly in "Barrier" method when using CPLEX as the solver. You can pass this option through `solve_options` arguments as below.

#### Tip (2): Case name
You can send your scenario for solving with a user-defined name. This name will be used to name the GDX files in GAMS, and it does not change the model/scenario name in your scenario database. You can leave the case name empty.

In [ ]:
# Solving the scenario (may up to 3 minutes)
casename = ('_').join([base.model, base.scenario, str(base.version)])
base.solve(solve_options={'lpmethod': '4'})

In [ ]:
# See the version of this scenario in your database
base.version

## 4.1. Visualizing output results
In this part, we can visualize some output results both on the sub-annual and yearly basis.

In [ ]:
from plotter import plotter
plotter("Korea baseline", base, "Korea", path + "//data/results")

In [ ]:
# Close the database at the end of your work
mp.close_db()